## Reserve and configure resources on KVM

Before you run this experiment, you will:

-   define the specific configuration of resources you need.
-   “instantiate” an experiment with your reserved resources.
-   wait for your resources to be configured.
-   log in to resources to carry out the experiment.

This exercise will guide you through those steps.

This notebook assumes you have completed the steps on https://teaching-on-testbeds.github.io/hello-chameleon/ and have uploaded the neccessary public key on the "key pairs" tab on the KVM@TACC site 

### Configure environment

In [2]:
import openstack, chi, chi.ssh, chi.network, chi.server, os
from chi.clients import session

In this section, we configure the Chameleon Python client.

For this experiment, we’re going to use the KVM@TACC site, which we indicate below.

We also need to specify the name of the Chameleon “project” that this experiment is part of. The project name will have the format “CHI-XXXXXX”, where the last part is a 6-digit number, and you can find it on your [user dashboard](https://chameleoncloud.org/user/dashboard/).

In the cell below, replace the project ID with your own project ID, then run the cell.

In [3]:
PROJECT_NAME = "CHI-231138"
chi.use_site("KVM@TACC")
chi.set("project_name", PROJECT_NAME)
os_conn = chi.clients.connection()
username = os.getenv('USER')

Now using KVM@TACC:
URL: https://kvm.tacc.chameleoncloud.org
Location: Austin, Texas, USA
Support contact: help@chameleoncloud.org


In [4]:
os_conn = chi.clients.connection()

### Define configuration for this experiment (1 VM)

For this specific experiment, we will need one virtual machines which will be of the `m1.medium` type, with 2 VCPUs, 4 GB memory, 40 GB disk space.

In [5]:
node_conf = [
 {'name': "node-0",  'flavor': 'm1.medium', 'image': 'CC-Ubuntu24.04', 'packages': ["python3", "virtualenv"], 'bastion': True}
]
net_conf = [
 {"name": "net0", "subnet": "192.168.1.0/24", "nodes": [{"name": "node-0",   "addr": "192.168.1.10"}]},
]
route_conf = []

### Configure resources

Now, we will prepare the VMs and network links that our experiment requires.

First, we will prepare a “public” network that we will use for SSH access to our VMs -

In [42]:
public_net = os_conn.network.create_network(name="public_net_" + username)
public_net_id = public_net.get("id")
public_subnet = os_conn.network.create_subnet(
    name="public_subnet_" + username,
    network_id=public_net.get("id"),
    ip_version='4',
    cidr="192.168.10.0/24",
    gateway_ip="192.168.10.1",
    is_dhcp_enabled = True
)

Next, we will prepare the “experiment” networks -

In [43]:
nets = []
net_ids = []
subnets = []
for n in net_conf:
    exp_net = os_conn.network.create_network(name="exp_" + n['name']  + '_' + username)
    exp_net_id = exp_net.get("id")
    os_conn.network.update_network(exp_net, is_port_security_enabled=False)
    exp_subnet = os_conn.network.create_subnet(
        name="exp_subnet_" + n['name']  + '_' + username,
        network_id=exp_net.get("id"),
        ip_version='4',
        cidr=n['subnet'],
        gateway_ip=None,
        is_dhcp_enabled = True
    )
    nets.append(exp_net)
    net_ids.append(exp_net_id)
    subnets.append(exp_subnet)

Now we create the VMs -

In [44]:
servers = []
server_ids = []
for i, n in enumerate(node_conf, start=10):
    print("printing:", i, n)
    image_uuid = os_conn.image.find_image(n['image']).id
    flavor_uuid = os_conn.compute.find_flavor(n['flavor']).id
    # find out details of exp interface(s)
    nics = [{'net-id': chi.network.get_network_id( "exp_" + net['name']  + '_' + username ), 'v4-fixed-ip': node['addr']} for net in net_conf for node in net['nodes'] if node['name']==n['name']]
    # also include a public network interface
    nics.insert(0, {"net-id": public_net_id, "v4-fixed-ip":"192.168.10." + str(i)})
    server = chi.server.create_server(
        server_name=n['name'] + "_" + username,
        image_id=image_uuid,
        flavor_id=flavor_uuid,
        nics=nics
    )
    servers.append(server)
    server_ids.append(chi.server.get_server(n['name'] + "_" + username).id)

printing: 10 {'name': 'node-0', 'flavor': 'm1.medium', 'image': 'CC-Ubuntu24.04', 'packages': ['python3', 'virtualenv'], 'bastion': True}


We wait for all servers to come up before we proceed -

In [45]:
for server_id in server_ids:
    chi.server.wait_for_active(server_id)

Next, we will set up SSH access to the VMs.

First, we will make sure the “public” network is connected to the Internet. Then, we will configure it to permit SSH access on port 22 for each port connected to this network.

In [46]:
# connect them to the Internet on the "public" network (e.g. for software installation)
router = chi.network.create_router('inet_router_' + username, gw_network_name='public')
chi.network.add_subnet_to_router(router.get("id"), public_subnet.get("id"))

{'network_id': '2ff3405d-9378-40bb-b588-066839d6abf8',
 'tenant_id': '13a1ac1ce275484caedc3394339486a1',
 'subnet_id': '8ab0b65d-767a-45eb-831e-28230a5aabdb',
 'subnet_ids': ['8ab0b65d-767a-45eb-831e-28230a5aabdb'],
 'port_id': '185d3027-1d9b-4db9-95f3-99f6d01d281f',
 'id': '62baaf84-995a-463f-a858-2dd906a853c1'}

In [47]:
# prepare SSH access on the servers that serve in "bastion" role
# WARNING: this relies on undocumented behavior of associate_floating_ip 
# that it associates the IP with the first port on the server
server_ips = []
for i, n in enumerate(node_conf):
    if 'bastion' in n and n['bastion']:
        ip = chi.server.associate_floating_ip(server_ids[i])
        server_ips.append(ip)

In [48]:
if not os_conn.get_security_group("Allow SSH"):
    os_conn.create_security_group("Allow SSH", "Enable SSH traffic on TCP port 22")
    os_conn.create_security_group_rule("Allow SSH", port_range_min=22, port_range_max=22, protocol='tcp', remote_ip_prefix='0.0.0.0/0')

security_group_id = os_conn.get_security_group("Allow SSH").id
for port in chi.network.list_ports(): 
    if port['port_security_enabled'] and port['network_id']==public_net.get("id"):
        os_conn.network.update_port(port['id'], security_groups=[security_group_id])

In [49]:
for ip in server_ips:
    chi.server.wait_for_tcp(ip, port=22)

The following cell may raise an error if some of your nodes are still getting set up! If that happens, wait a few minutes and try again. (And then a few minutes more, and try again, if it still raises an error.)

In [7]:
primary_remote = chi.ssh.Remote(server_ips[0])
physical_ips = [n['addr'] for n in net_conf[0]['nodes']]
server_remotes = [chi.ssh.Remote(physical_ip, gateway=primary_remote) for physical_ip in physical_ips]

Finally, we need to configure our resources, including software package installation

In [51]:
for i, n in enumerate(node_conf):
    # install packages
    if len(n['packages']):
            remote = server_remotes[i]
            remote.run(f"sudo apt update; sudo apt -y install " + " ".join(n['packages'])) 

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.26.118: b'0973d3549c0d2a8910162d8504d609b6'
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 192.168.1.10: b'0973d3549c0d2a8910162d8504d609b6'
  warnings.warn(




Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble InRelease [256 kB]
Get:2 http://security.ubuntu.com/ubuntu noble-security InRelease [126 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates InRelease [126 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu noble-backports InRelease [126 kB]
Get:5 http://security.ubuntu.com/ubuntu noble-security/main amd64 Packages [611 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu noble-updates/main amd64 Packages [833 kB]
Get:7 http://security.ubuntu.com/ubuntu noble-security/main amd64 Components [8964 B]
Get:8 http://security.ubuntu.com/ubuntu noble-security/main amd64 c-n-f Metadata [5892 B]
Get:9 http://security.ubuntu.com/ubuntu noble-security/universe amd64 Packages [802 kB]
Get:10 http://security.ubuntu.com/ubuntu noble-security/universe amd64 Components [52.0 kB]
Get:11 http://security.ubuntu.com/ubuntu noble-security/universe amd64 c-n-f Metadata [13.5 kB]
Get:12 http://security.ubuntu.com/ubuntu noble-sec

Reading package lists...
Building dependency tree...
Reading state information...
python3 is already the newest version (3.12.3-0ubuntu2).
python3 set to manually installed.
The following additional packages will be installed:
  python3-distlib python3-filelock python3-platformdirs python3-virtualenv
  python3-wheel-whl
Recommended packages:
  python3-distutils
The following NEW packages will be installed:
  python3-distlib python3-filelock python3-platformdirs python3-virtualenv
  python3-wheel-whl virtualenv
0 upgraded, 6 newly installed, 0 to remove and 154 not upgraded.
Need to get 486 kB of archives.
After this operation, 1796 kB of additional disk space will be used.
Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 python3-distlib all 0.3.8-1 [318 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu noble/universe amd64 python3-filelock all 3.13.1-1 [10.8 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu noble/main amd64 python3-platformdirs all 4.2.

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


Fetched 486 kB in 0s (996 kB/s)
Selecting previously unselected package python3-distlib.
(Reading database ... 89422 files and directories currently installed.)
Preparing to unpack .../0-python3-distlib_0.3.8-1_all.deb ...
Unpacking python3-distlib (0.3.8-1) ...
Selecting previously unselected package python3-filelock.
Preparing to unpack .../1-python3-filelock_3.13.1-1_all.deb ...
Unpacking python3-filelock (3.13.1-1) ...
Selecting previously unselected package python3-platformdirs.
Preparing to unpack .../2-python3-platformdirs_4.2.0-1_all.deb ...
Unpacking python3-platformdirs (4.2.0-1) ...
Selecting previously unselected package python3-wheel-whl.
Preparing to unpack .../3-python3-wheel-whl_0.42.0-2_all.deb ...
Unpacking python3-wheel-whl (0.42.0-2) ...
Selecting previously unselected package python3-virtualenv.
Preparing to unpack .../4-python3-virtualenv_20.25.0+ds-2_all.deb ...
Unpacking python3-virtualenv (20.25.0+ds-2) ...
Selecting previously unselected package virtualenv.
Pr

debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

Running kernel seems to be up-to-date.

No services need to be restarted.

No containers need to be restarted.

No user sessions are running outdated binaries.

No VM guests are running outdated hypervisor (qemu) binaries on this host.


### Get SSH login details

At this point, we should be able to log in to our head node over SSH! Run the following cell, and observe the output - you will see SSH command for the head node using the public ip.

In [52]:
print("ssh cc@" + server_ips[0])

ssh cc@129.114.26.118


Now, you can open an SSH session as follows:

-   In Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   Copy the SSH command from the output above, and paste it into the terminal.

Alternatively, you can use your local terminal to log on to the node, if you prefer. (On your local terminal, you may need to also specify your key path as part of the SSH command, using the `-i` argument followed by the path to your private key.)

For example ssh -i ~/.ssh/id_rsa_chameleon cc@public_ip

## Uploading data to objStore

First we want to install certain packages on the node including swift we will do this using a python virtualenv

In [53]:
primary_remote.run("virtualenv -p python3 myenv")

created virtual environment CPython3.12.3.final.0-64 in 705ms
  creator CPython3Posix(dest=/home/cc/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/home/cc/.local/share/virtualenv)
    added seed packages: pip==24.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


<Result cmd='virtualenv -p python3 myenv' exited=0>

In [54]:
primary_remote.run("source myenv/bin/activate; pip install python-swiftclient; pip install python-keystoneclient")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.3/145.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.6/341.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108

<Result cmd='source myenv/bin/activate; pip install python-swiftclient; pip install python-keystoneclient' exited=0>

First we will create a sample text file and upload + download it from the object store to test it.

In [55]:
primary_remote.run("echo 'This is a sample text file for the CHI@TACC object store.' > sample-file.txt")

<Result cmd="echo 'This is a sample text file for the CHI@TACC object store.' > sample-file.txt" exited=0>

You can access the Object Store from instances running on CHI@TACC and CHI@UC. Each region has its own store, meaning that objects uploaded to one are not visible to the other. In general you should use the store local to the region where your instances are running for the best performance. Additionally, you can also access the Object Store from the CHI@TACC or CHI@UC web interfaces under the Object Store panel.

#### Since we are using the KVM@TACC site which does not have an Object Store we will have to use the CHI@TACC Object Store along with application credentials and then use an OpenStack RC file to acces the Obj Store. ####

Follow the instructions on the link below to create your application credentials:

https://chameleoncloud.readthedocs.io/en/latest/technical/cli.html#creating-an-application-credential

Alternatively go to:
https://chameleoncloud.org/user/dashboard/ > Experiment > CHI@TACC > Identity > Apllication Credentials > + Create Application Credentials

Use the following to populate it:

Name: "batch_data_artifact"

Secret: here you can set your own password

Roles: click on "member" 

and then click on "Create Application credentials" button

You will then be prompted with a dialogue box to "Download openrc file"

Thenn scp the openrc file into your node using the scp command below

replace the path with your path


In [59]:
# scp from local system
print(f'scp -i ~/.ssh/id_rsa_chameleon /path/to/app-cred-batch_data_artifact-openrc.sh cc@{server_ips[0]}:/home/cc/')


Alternatively upload the openrc file here, open up a terminal via file > new > terminal and scp it to your node replace the path in the command below

In [57]:
# scp fter uploading file to chameleon
print(f'scp /path/to/app-cred-batch_data_artifact-openrc.sh cc@{server_ips[0]}:/home/cc/')

scp /path/to/app-cred-batch_data_artifact-openrc.sh cc@129.114.26.118:/home/cc/


In [9]:
container_name = "batch-data-container" #This is the container we created in the previous step: create_objStore.ipynb
#Upload the file to ObjStore
primary_remote.run('source myenv/bin/activate; source app-cred-batch_data_artifact-openrc.sh && swift --os-auth-type v3applicationcredential upload '+container_name+' sample-file.txt')

In [61]:
primary_remote.run("rm sample-file.txt; ls")

app-cred-batch_data_artifact-openrc.sh
myenv
openrc


<Result cmd='rm sample-file.txt; ls' exited=0>

In [62]:
#Download the file from ObjStore
primary_remote.run('source myenv/bin/activate; source app-cred-batch_data_artifact-openrc.sh && swift --os-auth-type v3applicationcredential download '+container_name+' sample-file.txt; ls')

sample-file.txt [auth 0.312s, headers 0.442s, total 0.443s, 0.000 MB/s]
app-cred-batch_data_artifact-openrc.sh
myenv
openrc
sample-file.txt


<Result cmd='source myenv/bin/activate; source app-cred-batch_data_artifact-openrc.sh && swift --os-auth-type v3applicationcredential download batch-data-container sample-file.txt; ls' exited=0>

In [66]:
#Delete the sample file from the ObjStore
primary_remote.run('source myenv/bin/activate; source app-cred-batch_data_artifact-openrc.sh && swift --os-auth-type v3applicationcredential delete '+container_name+' sample-file.txt')

sample-file.txt


<Result cmd='source myenv/bin/activate; source app-cred-batch_data_artifact-openrc.sh && swift --os-auth-type v3applicationcredential delete batch-data-container sample-file.txt' exited=0>

## Download the data

I this experiment we will be using: 

Airline On-Time Performance Data from the US Bureau of Transportation Statistics (BTS):

For nearly 40 years, all major US air carriers have been required to file statistics about each of their domestic flights with the BTS. The data they are required to file includes the scheduled departure and arrival times as well as the actual departure and arrival times. From the scheduled and actual arrival times, the arrival delay associated with each flight can be calculated. Therefore, this dataset can give us the true value or “label” for building a model to predict arrival delay.

In [28]:
# We will be using a bash script to download the data 
# git wgets the bash script and 
primary_remote.run("wget ...")
primary_remote.run("chmod +x download_bts_data.sh")
primary_remote.run("./download_bts_data.sh")

KeyboardInterrupt: 

In [10]:
folder_path = "./bts_data"
primary_remote.run('source myenv/bin/activate; source app-cred-batch_data_artifact-openrc.sh &&swift --os-auth-type v3applicationcredential upload '+container_name+' '+folder_path+'')

/opt/conda/lib/python3.10/site-packages/paramiko/client.py:889: UserWarning: Unknown ssh-ed25519 host key for 129.114.26.118: b'0973d3549c0d2a8910162d8504d609b6'
  warnings.warn(


bts_data/2023_2.zip
bts_data/2023_4.zip
bts_data/2023_12.zip
bts_data/2023_11.zip
bts_data/2023_1.zip
bts_data/2023_5.zip
bts_data/2023_7.zip
bts_data/2023_10.zip
bts_data/2023_9.zip
bts_data/2023_3.zip
bts_data/2023_8.zip
bts_data/2023_6.zip


<Result cmd='source myenv/bin/activate; source app-cred-batch_data_artifact-openrc.sh &&swift --os-auth-type v3applicationcredential upload batch-data-container ./bts_data' exited=0>

In [ ]:
#Downloading airport location and timezone data
primary_remote.run("wget https://raw.githubusercontent.com/GoogleCloudPlatform/data-science-on-gcp/652564b9feeeaab331ce27fdd672b8226ba1e837/04_streaming/simulate/airports.csv.gz")
primary_remote.run('source myenv/bin/activate; source app-cred-batch_data_artifact-openrc.sh &&swift --os-auth-type v3applicationcredential upload '+container_name+' airports.csv.gz')

Now we will check if you are able to download the date into your node from the ObjStore

In [12]:
primary_remote.run("rm -rf bts_data; ls")

app-cred-batch_data_artifact-openrc.sh
download_bts_data.sh
myenv
openrc
sample-file.txt


<Result cmd='rm -rf bts_data; ls' exited=0>

In [16]:
DEST_DIR="./downloaded_bts_data" 
primary_remote.run('source myenv/bin/activate; source app-cred-batch_data_artifact-openrc.sh && swift --os-auth-type v3applicationcredential download '+container_name+' --prefix "bts_data/" --output-dir '+DEST_DIR+'; ls')

./downloaded_bts_data/bts_data/2023_9.zip [auth 0.321s, headers 0.751s, total 1.686s, 20.549 MB/s]
./downloaded_bts_data/bts_data/2023_5.zip [auth 0.330s, headers 0.767s, total 1.810s, 19.142 MB/s]
./downloaded_bts_data/bts_data/2023_2.zip [auth 0.606s, headers 0.841s, total 1.861s, 19.702 MB/s]
./downloaded_bts_data/bts_data/2023_8.zip [auth 0.323s, headers 0.751s, total 1.869s, 19.112 MB/s]
./downloaded_bts_data/bts_data/2023_1.zip [auth 0.399s, headers 0.815s, total 1.925s, 17.745 MB/s]
./downloaded_bts_data/bts_data/2023_10.zip [auth 0.322s, headers 0.787s, total 1.954s, 18.051 MB/s]
./downloaded_bts_data/bts_data/2023_7.zip [auth 0.600s, headers 0.855s, total 1.972s, 22.026 MB/s]
./downloaded_bts_data/bts_data/2023_6.zip [auth 0.592s, headers 0.871s, total 1.996s, 20.710 MB/s]
./downloaded_bts_data/bts_data/2023_11.zip [auth 0.594s, headers 0.901s, total 2.027s, 19.241 MB/s]
./downloaded_bts_data/bts_data/2023_4.zip [auth 0.679s, headers 0.951s, total 2.034s, 20.647 MB/s]
./downlo

<Result cmd='source myenv/bin/activate; source app-cred-batch_data_artifact-openrc.sh && swift --os-auth-type v3applicationcredential download batch-data-container --prefix "bts_data/" --output-dir ./downloaded_bts_data; ls' exited=0>

In [21]:
# Check the files on your node
primary_remote.run("cd downloaded_bts_data/bts_data; ls")

2023_1.zip
2023_10.zip
2023_11.zip
2023_12.zip
2023_2.zip
2023_3.zip
2023_4.zip
2023_5.zip
2023_6.zip
2023_7.zip
2023_8.zip
2023_9.zip


<Result cmd='cd downloaded_bts_data/bts_data; ls' exited=0>

## Release resources

Now that you are able to upload and download data onto your node you can release the resources and move onto the next section of this experiment

In [23]:
# delete the nodes
server_ids = [chi.server.get_server_id(n['name'] + "_" + username) for n in node_conf]
server_ips = [d['addr'] for s in server_ids for d in chi.server.show_server(s).addresses['public_net_' + username] if d['OS-EXT-IPS:type']=='floating']
for server_id in server_ids:
    chi.server.delete_server(server_id)

In [24]:
# release the floating IP addresses used for SSH
for server_ip in server_ips:
    ip_details = chi.network.get_floating_ip(server_ip)
    chi.neutron().delete_floatingip(ip_details["id"])

In [25]:
# delete the router used for public Internet access
router = chi.network.get_router("inet_router_" + username)
public_subnet = chi.network.get_subnet("public_subnet_" + username)
public_net = chi.network.get_network("public_net_" + username)
chi.network.remove_subnet_from_router(router.get("id"), public_subnet.get("id"))
chi.network.delete_router(router.get("id"))

()

In [26]:
# delete the public network
chi.network.delete_subnet(public_subnet.get('id'))
chi.network.delete_network(public_net.get("id"))

()

In [27]:
# delete the experiment networks
subnets = [chi.network.get_subnet("exp_subnet_" + n['name']  + '_' + username) for n in net_conf]
nets    = [chi.network.get_network("exp_" + n['name']  + '_' + username) for n in net_conf]
for subnet, net in zip(subnets, nets):
    chi.network.delete_subnet(subnet.get('id'))
    chi.network.delete_network(net.get('id'))